In [36]:
# From https://stackoverflow.com/a/5849861
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        print('Elapsed [{}]: {}s'.format(self.name, time.time() - self.tstart))


In [ ]:
def betti(C, verbose = False) :
	
	import itertools
	import numpy as np
	import networkx as nx
	from scipy.sparse import lil_matrix
	
	def DIAGNOSTIC(*params) :
		if verbose :
			print(*params)
	
	#
	# 1. Prepare maximal cliques
	#
	
	# Sort each maximal clique, make sure it's a tuple
	# Also, commit C to memory if necessary
	C = [tuple(sorted(c)) for c in C]
	
	DIAGNOSTIC("Number of maximal cliques: {} ({}M)".format(len(C), round(len(C) / 1e6)))
	
	#
	# 2. Enumerate all simplices
	#
	
	# S[k] will hold all k-simplices
	# S[k][s] is the ID of simplex s
	S = []
	for k in range(0, max(len(s) for s in C)) :
		# Get all (k+1)-cliques, i.e. k-simplices, from max cliques mc
		Sk = set(c for mc in C for c in itertools.combinations(mc, k+1))
		# Check that each simplex is in increasing order
		assert(all((list(s) == sorted(s)) for s in Sk))
		# Assign an ID to each simplex, in lexicographic order
		S.append(dict(zip(sorted(Sk), range(0, len(Sk)))))

		DIAGNOSTIC("Number of {}-simplices: {}".format(k, len(Sk)))
	
	# The cliques are redundant now
	del C
		
	# Euler characteristic
	ec = sum(((-1)**k * len(S[k])) for k in range(0, len(S)))
	
	DIAGNOSTIC("Euler characteristic:", ec)
		
	#
	# 3. Construct the boundary operators
	#
	
	# D[k] is the boundary operator
	#      from the k complex
	#      to the k-1 complex
	D = [None for _ in S]
		
	# D[0] maps to zero by definition
	D[0] = lil_matrix( (1, len(S[0])) )

	# Construct D[1], D[2], ...
	for k in range(1, len(S)) :
		D[k] = lil_matrix( (len(S[k-1]), len(S[k])) )
		SIGN = np.asmatrix([(-1)**i for i in range(0, k+1)]).transpose()

		for (ks, j) in S[k].items() :
			# Indices of all (k-1)-subsimplices s of the k-simplex ks
			I = [S[k-1][s] for s in sorted(itertools.combinations(ks, k))]
			D[k][I, j] = SIGN

	# The simplices are redundant now
	del S
	
	for (k, d) in enumerate(D) :
		DIAGNOSTIC("D[{}] has shape {}".format(k, d.shape))
	
	# Check that D[k-1] * D[k] is zero
	assert(all((0 == np.dot(D[k-1], D[k]).count_nonzero()) for k in range(1, len(D))))
	
    
	#
	# 4. Compute rank and dimker of the boundary operators
	#
	
	# Compute rank using matrix SVD
	rk = [np.linalg.matrix_rank(d.todense()) for d in D] + [0]
	# Compute dimker using rank-nullity theorem
	ns = [(d.shape[1] - rk[n]) for (n, d) in enumerate(D)] + [0]

	# The boundary operators are redundant now
	#del D

	DIAGNOSTIC("rk:", rk)
	DIAGNOSTIC("ns:", ns)

	#
	# 5. Infer the Betti numbers
	#
	
	# Betti numbers
	# B[0] is the number of connected components
	B = [(n - r) for (n, r) in zip(ns[:-1], rk[1:])]

	# Remove trailing zeros
	while B and (B[-1] == 0) : B.pop()

	DIAGNOSTIC("Betti numbers:", B)
	
	# Check: Euler-Poincare formula (see Eqn 16 in [Zomorodian])
	epc = sum(((-1)**k * B[k]) for k in range(0, len(B)))
	if (ec != epc) : print("Warning: ec = {} and epc = {} should be equal".format(ec, epc))

	return (B, D)

In [196]:
# Compute the Betti numbers of a graph over Z/2Z
# 
# If G is a netwokx graph then
# C = networkx.find_cliques(G)
# (enumerates maximal cliques)
#
# RA, 2017-11-08 (CC-BY-4.0)
#
# Ref: 
#   A. Zomorodian, Computational topology (Notes), 2009
#   http://www.ams.org/meetings/short-courses/zomorodian-notes.pdf
#
# The computation of the rank is adapted from
#   https://triangleinequality.wordpress.com/2014/01/23/computing-homology/
# see also
#   https://gist.github.com/numpde/9584779ad235c6ee19be7a6bb87e8af5
#
# Gist for this function:
#   https://gist.github.com/numpde/a3f3476e601041da3cb968f31e95409d
#
def betti_bin(C) :
    from itertools import combinations as subcliques
    
    # Each maximal clique as a sorted tuple
    C = [tuple(sorted(mc)) for mc in C]
    
    # Betti numbers
    B = []
    
    # (k-1)-chain group
    Sl = dict()
    
    # Iterate over the dimension
    for k in range(0, max(len(s) for s in C)) :
        
        # Get all (k+1)-cliques, i.e. k-simplices, from max cliques mc
        Sk = set(c for mc in C for c in subcliques(mc, k+1))
        # Check that each simplex is in increasing order
        assert(all((list(s) == sorted(s)) for s in Sk))
        # Assign an ID to each simplex, in lexicographic order
        # (This ordering makes subsequent computations faster)
        Sk = dict(zip(sorted(Sk), range(0, len(Sk))))
        
        # Sk is now a representation of the k-chain group
        
        # "Default" Betti number (if rank is 0)
        B.append(len(Sk))
        
        # J2I is a mapped representation of the boundary operator
        # (Understood to have boolean entries instead of +1 / -1)
        
        J2I = {
            j : set(Sl[s] for s in subcliques(ks, k) if s)
            for (ks, j) in Sk.items()
        }
        
        Sl = Sk
        
        # Transpose J2I
        
        I2J = {
            i : set()
            for I in J2I.values() for i in I
        }
        
        for (j, I) in J2I.items() :
            for i in I :
                I2J[i].add(j)
                
        # Compute the rank of the boundary operator
        # The rank = The # of completed while loops
        # It's usually the most time-consuming part
        while J2I and I2J :
            # Default pivot option
            I = next(iter(J2I.values()))
            J = I2J[next(iter(I))]
            
            # An alternative option
            JA = next(iter(I2J.values()))
            IA = J2I[next(iter(JA))]
            
            # Choose the option with fewer indices
            # (reducing the number of loops below)
            if ((len(IA) + len(JA)) < (len(I) + len(J))) : (I, J) = (IA, JA)

            for i in I : 
                I2J[i] = J.symmetric_difference(I2J[i])
                if not I2J[i] : del I2J[i]

            for j in J : 
                J2I[j] = I.symmetric_difference(J2I[j])
                if not J2I[j] : del J2I[j]
            
            # Let
            #   D_k : C_k --> C_{k-1}
            # be the boundary operator. Recall that
            #   H_k = ker D_k / im D_{k+1}

            # Rank of  D_k  increases, therefore:
            B[k-1] -= 1
            # Nullspace dimension of  D_k  decreases, therefore:
            B[k]   -= 1
    
    # Remove trailing zeros
    while B and (B[-1] == 0) : B.pop()
    
    return B

In [192]:
import networkx as nx
import numpy as np


for i in range(0, 10) :
    G = nx.gnp_random_graph(33, 0.33, seed=i)
    B1 = betti(nx.find_cliques(G))[0]
    B2 = betti_bin(nx.find_cliques(G))
    print(B1 == B2)

for i in range(0, 10) :
    print("")
    
    G = nx.gnp_random_graph(40, 0.6, seed=i)
        
    with Timer('betti_bin') :
        B = betti_bin(nx.find_cliques(G))
        print(B)
    

True
True
True
True
True
True
True
True
True
True

[1, 0, 0, 27, 11]
Elapsed [betti_bin]: 0.922168493270874s

[1, 0, 0, 28, 8]
Elapsed [betti_bin]: 1.2159948348999023s

[1, 0, 0, 33, 2]
Elapsed [betti_bin]: 1.0952188968658447s

[1, 0, 0, 21, 5]
Elapsed [betti_bin]: 1.559732437133789s

[1, 0, 3, 15, 9]
Elapsed [betti_bin]: 1.0470771789550781s

[1, 0, 0, 11, 7]
Elapsed [betti_bin]: 2.420161724090576s

[1, 0, 2, 18, 6]
Elapsed [betti_bin]: 1.5180752277374268s

[1, 0, 0, 14, 25]
Elapsed [betti_bin]: 3.16487193107605s

[1, 0, 1, 31, 1]
Elapsed [betti_bin]: 0.857311487197876s

[1, 0, 0, 22, 17]
Elapsed [betti_bin]: 1.7307684421539307s


In [313]:
# Find the rank of a binary matrix over Z/2Z
# (conceptual implementation)
#
# RA, 2017-11-07 (CC-BY-4.0)
#
# Adapted from
#   https://triangleinequality.wordpress.com/2014/01/23/computing-homology/
#
def binary_rank(M) :
    
    # M-pty matrix?
    if not M.count_nonzero() : return 0

    # Find any nonzero entry, i.e. the pivot
    (p, q) = tuple(a[0] for a in M.nonzero())

    # Indices of entries to flip
    # (Could filter out p and q)
    I = M[:, q].nonzero()[0]
    J = M[p, :].nonzero()[1]

    # Flip those entries
    for i in I :
        for j in J :
            M[i, j] = not M[i, j]

    # Zero out pivot row p / column q
    # (Or delete them from the matrix)
    M[p, :] = 0
    M[:, q] = 0
    
    return 1 + binary_rank(M)

In [289]:
def binary_rank2(M) :
    # M-pty matrix?
    if not M.count_nonzero() : return 0
    
    # Convert to dict of sets
    I2J = dict((i, set(M[i, :].nonzero()[1])) for i in range(0, M.shape[0]))
    J2I = dict((j, set(M[:, j].nonzero()[0])) for j in range(0, M.shape[1]))

    I2J = { k : v for (k, v) in I2J.items() if v }
    J2I = { k : v for (k, v) in J2I.items() if v }
        
    rank = 0
    
    def matrix_from(i2j) :
        import numpy as np
        
        M = np.zeros((1 + max(i2j.keys()),  max(1 + max(J) for J in i2j.values() if J)))
        
        for (i, J) in i2j.items() :
            M[i, list(J)] = 1
        
        return M
    
    with Timer('dict (effective)') :
        while J2I :
            I = next(iter(J2I.values()))
            J = I2J[next(iter(I))]

            for i in I : 
                delta = I2J[i].symmetric_difference(J)
                if delta :
                    I2J[i] = delta
                else :
                    del I2J[i]

            for j in J : 
                delta = J2I[j].symmetric_difference(I)
                if delta :
                    J2I[j] = delta
                else :
                    del J2I[j]

            rank += 1
    
    return rank

In [293]:
import networkx as nx
import numpy as np

for i in range(0, 10) :
    with Timer('Gnp') :
        G = nx.gnp_random_graph(54, 0.7, seed=i)
    
    
    with Timer('betti') :
        (B, D) = betti(nx.find_cliques(G))
        d = D[2]
    
    print("Matrix size:", d.shape)
    
    with Timer('svd') :
        rank0 = np.linalg.matrix_rank(d.todense())
    with Timer('matrix') :
        rank1 = binary_rank(abs(d).tolil())
    with Timer('dict (total)') :
        rank2 = binary_rank2(abs(d).tolil())
        
    print("")
    
    print("Ranks agree:", rank0 == rank1, "/", rank0 == rank2)
    print("")

[Gnp]
Elapsed: 0.001171112060546875s
[betti]
Elapsed: 16.089121341705322s
Matrix size: (856, 6659)
[svd]
Elapsed: 0.4468040466308594s
[matrix]
Elapsed: 9.731476783752441s
[dict (effective)]
Elapsed: 0.9269750118255615s
[dict (total)]
Elapsed: 6.616172552108765s

Ranks agree: True / True

[Gnp]
Elapsed: 0.0009338855743408203s
[betti]
Elapsed: 19.49002695083618s
Matrix size: (846, 6502)
[svd]
Elapsed: 0.42658519744873047s
[matrix]
Elapsed: 10.619252920150757s
[dict (effective)]
Elapsed: 0.9568219184875488s
[dict (total)]
Elapsed: 6.379350423812866s

Ranks agree: True / True

[Gnp]
Elapsed: 0.0011792182922363281s
[betti]
Elapsed: 17.216083765029907s
Matrix size: (856, 6674)
[svd]
Elapsed: 0.44940853118896484s
[matrix]
Elapsed: 11.041736602783203s
[dict (effective)]
Elapsed: 0.8715791702270508s
[dict (total)]
Elapsed: 6.634899377822876s

Ranks agree: True / True

[Gnp]
Elapsed: 0.001077890396118164s
[betti]
Elapsed: 17.724985599517822s
Matrix size: (858, 6730)
[svd]
Elapsed: 0.429675102233